# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Loading Dataset

In [ ]:
!ls ../input/voicegender/voice.csv

In [ ]:
data = pd.read_csv("../input/voicegender/voice.csv")
df = pd.DataFrame(data)
df

# Data Pre-processing

In [ ]:
df.isnull().sum()

In [ ]:
df['label'].value_counts()

Data is also balanced

In [ ]:
df.info()

Sicne Lable is object so we need to convert it into numerical data

In [ ]:
la=LabelEncoder()
df['label']=la.fit_transform(df['label'])

In [ ]:
data.info()

In [ ]:
df['label']

So now it is converted into mumerical data type so now we can apply our classiffier

In [ ]:
corr=df.corr()
corr

# Data Visualization

In [ ]:
plt.subplots(figsize=(36,12))
sns.heatmap(corr, annot=True)

In [ ]:
corr['label'].sort_values(ascending=False)

Since the positive values are less so i will take all of them

# Splitting data into traning and testing

In [ ]:
x=df.drop(['meanfun','Q25','meanfreq','centroid','median','maxdom','mindom','dfrange','meandom','mode','maxfun','minfun','label'], axis=1)
x.describe()

In [ ]:
scaler = StandardScaler()
x = pd.DataFrame(data=scaler.fit_transform(x), columns=x.columns)
x

In [ ]:
y=df.iloc[:,-1]
y

In [ ]:
y.shape

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=0)

In [ ]:
cla=SVC()
cla.fit(xtrain, ytrain)

In [ ]:
cla.get_params()

# Data Training Using GridSearch

In [ ]:
# Fitting Data to find best parameters
param = {
    'C': [1,2],
    'kernel': ['rbf', 'sigmoid'],
    'degree' : [3,4],
    'break_ties' : [True, False],
    'verbose' : [True, False],
    'probability' : [True, False]
}
grid = GridSearchCV(estimator=cla, param_grid=param, cv= 5)
grid.fit(xtrain, ytrain)

In [ ]:
# Finding best parameters
grid.best_params_

# Applying Classifier

In [ ]:
# Data fitting using parameters found from GridSearch
cla2=SVC(C=2 , random_state=0, kernel= 'rbf', degree=3 , break_ties= True, verbose=True , probability=True )
cla2.fit(xtrain, ytrain)

# Checking Accuracy

In [ ]:
ytest_predict= cla2.predict(xtest)
accuracy_score(ytest_predict, ytest)

In [ ]:
# ROC Curve

fpr, tpr, thresholds =roc_curve(ytest, ytest_predict, pos_label=1)
plt.plot(fpr, tpr)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()
# Measure the area under the curve.  The closer to 1, the "better" the predictions.
print("AUC of the predictions: {0}".format(auc(fpr, tpr)))

# Measure the Accuracy Score
print("Accuracy score of the predictions: {0}".format(accuracy_score(ytest_predict, ytest)))


In [ ]:
# Confusion matrix

confu = confusion_matrix(ytest, ytest_predict, labels = [0,1])
sns.heatmap(confu, annot=True)

In [ ]:
print("Classification Report for our model is ")
print(classification_report(ytest, ytest_predict))